### Imports

In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

## Redução de linhas por modelo

## Tempo medio de cada modelo por fontes

#

## Precisão dos modelos por fontes

### Inferências

In [5]:
import pandas as pd
import json
import glob
import re
from pathlib import Path

def extract_analysis_data(analysis_str):
    """Extrai o JSON da string analysis (que pode conter tags <think>)"""
    match = re.search(r'\{.*"CLASSIFICATION":.*"CONFIDENCE":.*\}', analysis_str, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0).replace('\n', ''))
        except:
            return None
    return None

# Encontrar todos os arquivos JSONL no padrão especificado
files = glob.glob('inference/**/*.jsonl', recursive=True)

data = []

for file_path in files:
    # Extrair o modelo do caminho do arquivo
    path = Path(file_path)
    model = path.parts[1]  # Extrai o segundo componente do caminho (inference/{model}/...)
    font = path.parts[2]
    type = path.parts[3]
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                analysis_data = extract_analysis_data(record['analysis'])
                
                if analysis_data:
                    data.append({
                        'model': model,
                        'type': type,
                        'font': font,
                        'source_file': record['file'],
                        'classification': analysis_data['CLASSIFICATION'],
                        'confidence': analysis_data['CONFIDENCE'],
                        'justification': analysis_data['JUSTIFICATION']
                    })
                    
            except Exception as e:
                print(f"Erro ao processar linha do arquivo {file_path}: {str(e)}")

# Criar DataFrame
df = pd.DataFrame(data)

# Exibir estrutura do DataFrame
print("-----------------------------------")
print(f"Total de registros processados: {len(df)}")
print("\nPrimeiros registros:")
display(df.head())
df.to_csv("classifications.csv", index=False)

Erro ao processar linha do arquivo inference\gemma3_12b\data\safe\safe_20250505_195500.jsonl: 'JUSTIFICATION'
Erro ao processar linha do arquivo inference\llama3.1\data\safe\safe_20250505_195200.jsonl: 'JUSTIFICATION'
Erro ao processar linha do arquivo inference\qwen3_14b\data\attack\20250411_180254.jsonl: 'JUSTIFICATION'
Erro ao processar linha do arquivo inference\qwen3_14b\data\attack\20250411_180711.jsonl: 'JUSTIFICATION'
Erro ao processar linha do arquivo inference\qwen3_14b\data\safe\safe_20250505_185900.jsonl: 'JUSTIFICATION'
-----------------------------------
Total de registros processados: 969

Primeiros registros:


,model,type,font,source_file,classification,confidence,justification
0,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.95,The batch contains multiple events indicating ...
1,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.75,The batch includes multiple registry modificat...
2,deepseek-r1_14b,attack,data,data/attack/20250411_175806.jsonl,INTERESTING,0.95,The batch shows multiple instances of powershe...
3,deepseek-r1_14b,attack,data,data/attack/20250411_175826.jsonl,INTERESTING,0.85,The batch contains multiple events involving P...
4,deepseek-r1_14b,attack,data,data/attack/20250411_175826.jsonl,NOT INTERESTING,0.95,Most events involve legitimate processes like ...


### Métricas

### Gerando inferências concretas

In [6]:
import pandas as pd

def process_group(group):
    model = group['model'].iloc[0]
    source_file = group['source_file'].iloc[0]

    if len(group) == 1:
        result = group[['type', 'font', 'classification', 'confidence', 'justification']].copy()
        result['model'] = model
        result['source_file'] = source_file
        return result
    else:
        class_counts = group['classification'].value_counts(normalize=True)
        majority_class = class_counts.idxmax()
        majority_ratio = class_counts.max()

        if majority_ratio > 0.5:
            consolidated = pd.DataFrame({
                'type': [group['type'].iloc[0]],
                'font': [group['font'].iloc[0]],
                'classification': [majority_class],
                'confidence': [group['confidence'].mean()],
                'justification': [' | '.join(group['justification'].unique())],
                'model': [model],
                'source_file': [source_file]
            })
            return consolidated
        else:
            result = group[['type', 'font', 'classification', 'confidence', 'justification']].copy()
            result['model'] = model
            result['source_file'] = source_file
            return result


# Carregar dados
df = pd.read_csv('classifications.csv')

# Processar grupos e concatenar resultados
processed = (
    df.groupby(['model', 'source_file'], group_keys=False)
    .apply(process_group)
    .reset_index()
)

# Selecionar e reordenar colunas
final_df = processed[['model', 'font', 'source_file', 'type', 'classification', 'confidence', 'justification']]

# Salvar resultado
final_df.to_csv('consolidated_classifications.csv', index=False)
print("Consolidação concluída com sucesso!")
print("\nExemplo do resultado:")
display(final_df.head())

Consolidação concluída com sucesso!

Exemplo do resultado:


C:\Users\lucas\AppData\Local\Temp\ipykernel_21972\3620560010.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(process_group)


,model,font,source_file,type,classification,confidence,justification
0,deepseek-r1_14b,data,data/attack/20250411_175749.jsonl,attack,INTERESTING,0.85,The batch contains multiple events indicating ...
1,deepseek-r1_14b,data,data/attack/20250411_175806.jsonl,attack,INTERESTING,0.95,The batch shows multiple instances of powershe...
2,deepseek-r1_14b,data,data/attack/20250411_175826.jsonl,attack,INTERESTING,0.85,The batch contains multiple events involving P...
3,deepseek-r1_14b,data,data/attack/20250411_175826.jsonl,attack,NOT INTERESTING,0.95,Most events involve legitimate processes like ...
4,deepseek-r1_14b,data,data/attack/20250411_175855.jsonl,attack,NOT INTERESTING,0.40,The events include normal user activity such a...


### Gerando métricas finais

In [7]:
import numpy as np

final_df["false_positive"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["false_negative"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

final_df["true_positive"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["true_negative"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

display(final_df.head())
final_df.to_csv("consolidated_classifications.csv", index=False)

,model,font,source_file,type,classification,confidence,justification,false_positive,false_negative,true_positive,true_negative
0,deepseek-r1_14b,data,data/attack/20250411_175749.jsonl,attack,INTERESTING,0.85,The batch contains multiple events indicating ...,0,0,1,0
1,deepseek-r1_14b,data,data/attack/20250411_175806.jsonl,attack,INTERESTING,0.95,The batch shows multiple instances of powershe...,0,0,1,0
2,deepseek-r1_14b,data,data/attack/20250411_175826.jsonl,attack,INTERESTING,0.85,The batch contains multiple events involving P...,0,0,1,0
3,deepseek-r1_14b,data,data/attack/20250411_175826.jsonl,attack,NOT INTERESTING,0.95,Most events involve legitimate processes like ...,0,1,0,0
4,deepseek-r1_14b,data,data/attack/20250411_175855.jsonl,attack,NOT INTERESTING,0.40,The events include normal user activity such a...,0,1,0,0


In [8]:
import pandas as pd

# Carregar o CSV
df = pd.read_csv("consolidated_classifications.csv")

# Função para calcular métricas por grupo (modelo + fonte)
def calcular_metricas(grupo):
    y_true = grupo["true_positive"] + grupo["true_negative"] > 0  # se for verdadeiro positivo ou negativo
    y_pred = ~grupo["false_positive"].astype(bool)  # predição correta se não for falso positivo

    # Isso assume que estamos interessados na predição de INTERESTING como positivo
    tp = grupo["true_positive"].sum()
    tn = grupo["true_negative"].sum()
    fp = grupo["false_positive"].sum()
    fn = grupo["false_negative"].sum()

    total = tp + tn + fp + fn

    acc = (tp + tn) / total if total else 0
    prec = tp / (tp + fp) if (tp + fp) else 0
    rec = tp / (tp + fn) if (tp + fn) else 0
    f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) else 0

    return pd.Series({
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    })

# Aplicar por modelo + fonte
metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()

# Exibir
display(metricas_por_grupo)
metricas_por_grupo.to_csv("result_table.csv", decimal=',', sep=';', index=False)


C:\Users\lucas\AppData\Local\Temp\ipykernel_21972\3569469697.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()


,model,font,accuracy,precision,recall,f1_score
0,deepseek-r1_14b,data,0.760000,0.884615,0.718750,0.793103
1,gemma3_12b,data,0.666667,0.653061,1.000000,0.790123
2,llama3.1,data,0.627451,0.627451,1.000000,0.771084
3,mistral-nemo,data,0.673077,0.666667,0.969697,0.790123
4,phi4,data,0.716981,0.680851,1.000000,0.810127
5,qwen3_14b,data,0.611111,0.677419,0.656250,0.666667


## Comitês

In [ ]:
df_classification = pd.read_csv("consolidated_classifications.csv")
df_result = pd.read_csv("result_table.csv", sep=';', decimal=',')
df_result.sort_values("f1_score", ascending=False)




,model,font,accuracy,precision,recall,f1_score
4,phi4,data,0.716981,0.680851,1.000000,0.810127
0,deepseek-r1_14b,data,0.760000,0.884615,0.718750,0.793103
1,gemma3_12b,data,0.666667,0.653061,1.000000,0.790123
3,mistral-nemo,data,0.673077,0.666667,0.969697,0.790123
2,llama3.1,data,0.627451,0.627451,1.000000,0.771084
5,qwen3_14b,data,0.611111,0.677419,0.656250,0.666667
